In [1]:
import pandas as pd
import numpy as np
import random
import torch
import transformers
import torch.nn as nn
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_metric, Dataset
from sklearn.metrics import classification_report, f1_score

In [2]:
!pip install transformers datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 1.7 MB/s eta 0:00:00a 0:00:01


In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
train = pd.read_csv('../data/first_task/train.csv')
test = pd.read_csv('../data/first_task/test.csv')
sample = pd.read_csv('../data/first_task/sample_submission.csv')

In [5]:
from sklearn.model_selection import train_test_split

train_text, test_text, train_label, test_label = train_test_split(train['text'], train['sentiment'], test_size=0.2,
                                                                  random_state=42)


In [6]:
from transformers import AutoTokenizer

model_ckpt = "papluca/xlm-roberta-base-language-detection"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)



tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [13]:
def preprocess_function(text):
    return tokenizer(text, truncation=True)

In [16]:
tokenized_train = train['text'].apply(preprocess_function)

In [17]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [20]:
import evaluate

f1 = evaluate.load("f1")

In [21]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels)

In [22]:
id2label = {0: "NEUTRAL", 1: "POSITIVE", 2: "NEGATIVE"}
label2id = {"NEUTRAL": 0, "POSITIVE": 1, "NEGATIVE": 2}

In [24]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt, num_labels=3, id2label=id2label, label2id=label2id,ignore_mismatched_sizes=True)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at papluca/xlm-roberta-base-language-detection and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([20]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([20, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [35]:
!pip install accelerate -U

Python(7075) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [36]:
training_args = TrainingArguments(
    output_dir="../models/bert.pkl",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

82633     Отель расположен очень удачно(на мой взгляд)-в...
54230     Великолепное обслуживание. Сотрудники ресепшен...
121119    Пора уже обнародовать скрываемые властями кост...
760       Коррупция в неврологическом отделении Я просто...
132950    отличный отель в современном стиле, есть небол...
                                ...                        
119879    Отмечали день рождения небольшой компании , вс...
103694    Номер был с окнами во двор, поэтому было тихо ...
131932    мы с подругой учимся рядом - потому и забегаем...
146867    Возможно, у нас бывает, что на внутренний рыно...
121958    Спасибо Прошли 2курса лечения на дневном стаци...
Name: text, Length: 151912, dtype: object

NameError: name 'tokens_train' is not defined

In [4]:
from sklearn.metrics import f1_score


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds)
    return {'F1': f1}

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',  #Выходной каталог
    num_train_epochs=3,  #Кол-во эпох для обучения
    per_device_train_batch_size=8,  #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size=8,  #Размер пакета для каждого устройства во время валидации
    weight_decay=0.01,  #Понижение весов
    logging_dir='./logs',  #Каталог для хранения журналов
    load_best_model_at_end=True,  #Загружать ли лучшую модель после обучения
    learning_rate=1e-5,  #Скорость обучения
    evaluation_strategy='epoch',  #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy='epoch',  #Логирование после каждой эпохи
    save_strategy='epoch',  #Сохранение после каждой эпохи
    save_total_limit=1,
    seed=21)

In [ ]:
trainer = Trainer(model=model,
                  tokenizer=tokenizer,
                  args=training_args,
                  train_dataset=train_dataset,
                  eval_dataset=train_dataset,
                  compute_metrics=compute_metrics)

In [ ]:
trainer.train()

In [ ]:
model_path = "fine-tune-bert"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

In [ ]:
def get_prediction():
    test_pred = trainer.predict(test_dataset)
    labels = np.argmax(test_pred.predictions, axis=-1)
    return labels


pred = get_prediction()